1. Загрузите данные из файла churn-bigml-80.csv
Проведите базовый осмотр данных.
сколько строк, сколько столбцов, есть ли пустые значения?

определите общее количество абонентов отказавшихся от обслуживания (переменная Churn)

Исследуйте зависимость целевой переменной Churn 
от переменных 
Voice mail plan
International plan
Customer service calls



Сформулируйте гипотезы о зависимости.

In [151]:
import pandas as pd
import numpy as np
import seaborn as sns

Загрузим датасет

In [152]:
df = pd.read_csv("churn-bigml-80.csv")

In [153]:
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [154]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   2666 non-null   object 
 1   Account length          2666 non-null   int64  
 2   Area code               2666 non-null   int64  
 3   International plan      2666 non-null   object 
 4   Voice mail plan         2666 non-null   object 
 5   Number vmail messages   2666 non-null   int64  
 6   Total day minutes       2666 non-null   float64
 7   Total day calls         2666 non-null   int64  
 8   Total day charge        2666 non-null   float64
 9   Total eve minutes       2666 non-null   float64
 10  Total eve calls         2666 non-null   int64  
 11  Total eve charge        2666 non-null   float64
 12  Total night minutes     2666 non-null   float64
 13  Total night calls       2666 non-null   int64  
 14  Total night charge      2666 non-null   

Датафрейм содержит 19 признаков и один целевой показатель (бинарный). Всего у нас 2666 наблюдений. Пропусков нет.

Сразу заменим некоторые типы данных на числовые. Понизим разрядность чисел для уменьшения объема памяти

In [155]:
df["International plan"] = df["International plan"].map({"Yes": 1, "No": 0})
df["Voice mail plan"] = df["Voice mail plan"].map({"Yes": 1, "No": 0})
df["Churn"] = df["Churn"].map({True: 1, False: 0})

In [156]:
for col in (['Account length', 'Area code', 'International plan',
             'Voice mail plan', 'Number vmail messages',
             'Total day calls', 'Total eve calls', 'Total night calls', 
             'Total intl calls', 'Customer service calls', 
             'Customer service calls', 'Churn']):
    df[col] = df[col].astype("int32", copy=False)

for col in (['Total day minutes', 'Total day charge', 
             'Total eve minutes', 'Total eve charge', 
             'Total night minutes', 'Total night charge', 
             'Total intl minutes', 'Total intl charge']):
    df[col] = df[col].astype("float32", copy=False)


In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   2666 non-null   object 
 1   Account length          2666 non-null   int32  
 2   Area code               2666 non-null   int32  
 3   International plan      2666 non-null   int32  
 4   Voice mail plan         2666 non-null   int32  
 5   Number vmail messages   2666 non-null   int32  
 6   Total day minutes       2666 non-null   float32
 7   Total day calls         2666 non-null   int32  
 8   Total day charge        2666 non-null   float32
 9   Total eve minutes       2666 non-null   float32
 10  Total eve calls         2666 non-null   int32  
 11  Total eve charge        2666 non-null   float32
 12  Total night minutes     2666 non-null   float32
 13  Total night calls       2666 non-null   int32  
 14  Total night charge      2666 non-null   

Определим число ушедших абонентов

In [158]:
df['Churn'].agg(['sum', 'mean'])

sum     388.000000
mean      0.145536
Name: Churn, dtype: float64

Ушедших абонетов не более 15 %

In [159]:
df.groupby('Churn')['International plan'].mean()

Churn
0    0.066725
1    0.304124
Name: International plan, dtype: float64

Из оставшихся абонентов почти никто не использовал услугу International plan

In [160]:
df.groupby('Churn')['Voice mail plan'].mean()

Churn
0    0.293240
1    0.167526
Name: Voice mail plan, dtype: float64

Услугой Voice mail plan пользовалось в среднем в 2 раза больше среди оставшихся абонентов

In [161]:
df.groupby('Churn')['Customer service calls'].mean()

Churn
0    1.453029
1    2.206186
Name: Customer service calls, dtype: float64

Ушедшие клиенты в среднем примерно в 1.5 раза чаще свонили в службу поддержки